In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, IntegerType, StringType

# Spark session z obsługą Kafka
spark = SparkSession.builder \
    .appName("StructuredStreamingKafka") \
    .master("local[*]") \
    .config("spark.jars.packages", ",".join([
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1",
        "org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.4.1"
    ])) \
    .getOrCreate()
    
spark.sparkContext.setLogLevel("ERROR")

# Schemat danych JSON
schema = StructType() \
    .add("id", IntegerType()) \
    .add("value", StringType())

# Strumień z Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka_streaming_lab:9092") \
    .option("subscribe", "spark-lab1-topic") \
    .option("startingOffsets", "latest") \
    .load()

# Parsowanie danych
parsed_df = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Wypisz wynik na konsolę
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()
